異体字のペアを作成する

In [87]:
%cd itaiji

[Errno 2] No such file or directory: 'itaiji'
/home/analysis01/src/kanjikana-model/dict/itaiji


In [88]:
#%cd tobunken

In [89]:
#!wget --no-check-certificate -O index.html https://www.tobunken.go.jp/archives/%E7%95%B0%E4%BD%93%E5%AD%97%E3%83%AA%E3%82%B9%E3%83%88/

In [90]:
#!python extract.py --infile="index.html" --outfile=tobunken_itaiji.txt

In [91]:
#%cd ../

ペアにして、属性をつける。 

key1,key2:[attr1,attr2]
key2,key1:[attr1,attr2]      

In [92]:
pair={}

In [93]:
def make_pair(fname,attr):
    with open(fname,'r',encoding='utf-8') as f:
        for l in f:
            items=l.rstrip().split(',')
            keyA=f'{items[0]},{items[1]}'      
            keyB=f'{items[1]},{items[0]}'
            if keyA not in pair:
                pair[keyA]=[]
            if keyB not in pair:
                pair[keyB]=[]
            pair[keyA].append(attr)
            alst=pair[keyA]
            pair[keyB].append(attr)
            blst=pair[keyB]
            pair[keyA]=list(set(alst))
            pair[keyB]=list(set(blst))

In [94]:
#make_pair('tobunken/tobunken_itaiji.txt','tobunken_itaiji')

In [95]:
len(pair)

0

In [96]:
make_pair('../oss/mozc/itaiji.txt','mozc_itaiji')

In [97]:
len(pair)

1420

In [98]:
make_pair('../oss/skk/itaiji.txt','skk_itaiji')

In [99]:
len(pair)

2344

In [100]:
siz={}
for k,v in pair.items():
    if len(v) not in siz:
        siz[len(v)]=0
    siz[len(v)]+=1

In [101]:
siz

{1: 1112, 2: 1232}

同じ文字のグループを作成する

In [102]:
group={}  # group[kanji1]=groupid    漢字単体とグループ番号を作成

In [103]:
for k in pair.keys():
    key1,key2=k.split(',')
    group[key1]=-1
    group[key2]=-1

In [104]:
gid=0

In [105]:
for k in pair.keys():
    key1,key2=k.split(',')
    cand_gid=group[key1]
    if cand_gid<0:
        group[key2]
    if cand_gid<0: # newgroup
        group[key1]=gid
        group[key2]=gid
        gid+=1
    else:
        group[key1]=cand_gid
        group[key2]=cand_gid

In [106]:
gid

854

In [107]:
rev_group={}   # group[groupid]=[kanji1,kanji2,]    漢字単体とグループ番号を作成

In [108]:
for k,v in group.items():
    if v not in rev_group:
        rev_group[v]=[]
    rev_group[v].append(k)

In [109]:
len(rev_group)

790

In [110]:
rev_group

{0: ['叱', '𠮟'],
 1: ['埓', '埒'],
 2: ['髙', '高'],
 3: ['啮', '噛', '嚙'],
 4: ['噓', '嘘'],
 5: ['蟬', '蝉'],
 6: ['驒', '騨'],
 7: ['簞', '箪'],
 8: ['剝', '剥'],
 9: ['屛', '屏'],
 10: ['穎', '頴'],
 11: ['禰', '祢', '袮'],
 12: ['儘', '侭'],
 13: ['壺', '壷'],
 14: ['彎', '湾'],
 228: ['弯', '灣'],
 15: ['攪', '撹'],
 16: ['檜', '桧'],
 17: ['檮', '梼'],
 18: ['櫟', '檪'],
 19: ['濤', '涛'],
 20: ['灌', '潅', '濯'],
 21: ['濾', '沪', '瀘'],
 22: ['箏', '筝'],
 23: ['籠', '篭'],
 375: ['藪', '薮', '籔'],
 25: ['蘆', '芦'],
 26: ['蠣', '蛎'],
 27: ['蠅', '蝿'],
 28: ['諫', '諌'],
 29: ['賤', '賎'],
 30: ['邇', '迩'],
 763: ['靱'],
 31: ['靭', '靫'],
 32: ['頸', '頚'],
 33: ['鰺', '鯵'],
 34: ['鶯', '鴬'],
 35: ['俠', '侠'],
 36: ['啞', '唖'],
 37: ['囊', '嚢'],
 38: ['塡', '填'],
 39: ['搔', '掻'],
 40: ['潑'],
 244: ['溌', '醗', '発'],
 41: ['瀆', '涜'],
 42: ['焰', '焔'],
 43: ['禱', '祷'],
 44: ['繡'],
 675: ['繍', '綉'],
 45: ['繫', '繋'],
 46: ['萊', '莱'],
 47: ['蔣', '蒋'],
 48: ['蠟', '蝋'],
 49: ['軀', '躯'],
 50: ['醬', '醤'],
 51: ['醱', '發'],
 52: ['頰', '頬'],
 53: ['顚', '顛'],
 54:

第一水準から順に優先順をつけて、優先のものに代替するようにする
さらに小中高の順に優先する

In [111]:
def load_jis(fname):
    with open(fname,'r',encoding='utf-8') as f:
        hsh ={l.rstrip():1 for l in f}
    return hsh

In [112]:
dai1=load_jis('../daitai/jisdai1.dat')

In [113]:
dai2=load_jis('../daitai/jisdai2.dat')

In [114]:
dai3=load_jis('../daitai/jisdai3.dat')

In [115]:
dai4=load_jis('../daitai/jisdai4.dat')

In [116]:
syo=load_jis('../daitai/syogaku.dat')

In [117]:
cyu=load_jis('../daitai/cyugaku.dat')

In [118]:
kou=load_jis('../daitai/koukou.dat')

In [119]:
suijun=[syo,cyu,kou,dai1,dai2,dai3,dai4]

In [120]:
import regex
import re

In [121]:
def get_suijun(s):
    # 平仮名カタカナは最優先 0でかえす  旧字は除外
    if (regex.match('^\p{Script=Hiragana}+$',s) is not None or regex.match('^\p{Script=Katakana}+$',s) is not None) and re.match(f'^[ゐゑ	゚゛゜ゝゞゟ]',s) is None and re.match('^[ヰヱヸヹヽヾヿ]+$',s) is None:
        return 0 
    
    for i,sui in enumerate(suijun):
        if s in sui:
            return i+1
    return len(suijun) # 1-4水準外

In [122]:
def get_min_suijun(lst): # 配列内で最も水準が高いものの水準
    min_suijun=5
    min_moji=''
    for l in lst:
        sui=get_suijun(l)
        if min_suijun>sui:
            min_suijun=sui
            min_moji=l
    if min_suijun==5:
        min_moji=lst[0]  # 全部1ー4水準外の時は最初のものを選択
    return min_suijun,min_moji        

In [123]:
ngroup={}  # キーが水準の低いもののみ残す  {key2:key1,key3:key1,key4:key1}  key1 is min suijun

In [124]:
for vv in rev_group.values():
    min_sui,min_moji=get_min_suijun(vv)
    for val in vv:
        if val == min_moji:
            continue
        key=f'{min_moji},{val}'
        attr=[]
        if key in pair:
            attr=list(set(pair[key]))
        ngroup[val]={min_moji:{"dics":attr}}


In [125]:
len(ngroup)

897

In [126]:
ngroup

{'𠮟': {'叱': {'dics': ['mozc_itaiji']}},
 '埒': {'埓': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '髙': {'高': {'dics': ['mozc_itaiji']}},
 '啮': {'噛': {'dics': ['mozc_itaiji']}},
 '嚙': {'噛': {'dics': ['mozc_itaiji']}},
 '噓': {'嘘': {'dics': ['mozc_itaiji']}},
 '蟬': {'蝉': {'dics': ['mozc_itaiji']}},
 '驒': {'騨': {'dics': ['mozc_itaiji']}},
 '簞': {'箪': {'dics': ['mozc_itaiji']}},
 '剥': {'剝': {'dics': ['mozc_itaiji']}},
 '屏': {'屛': {'dics': ['mozc_itaiji']}},
 '頴': {'穎': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '祢': {'禰': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '袮': {'禰': {'dics': ['skk_itaiji']}},
 '儘': {'侭': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '壺': {'壷': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '彎': {'湾': {'dics': ['skk_itaiji']}},
 '灣': {'弯': {'dics': ['skk_itaiji']}},
 '攪': {'撹': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '檜': {'桧': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '檮': {'梼': {'dics': ['skk_itaiji', 'mozc_itaiji']}},
 '檪': {'櫟': {'dics': ['skk_itaiji', 'mozc_itaiji']}

In [127]:
import json

In [128]:
with open('../itaiji.json','w',encoding='utf-8') as f:
    json.dump(ngroup,f,ensure_ascii=False,indent=2)

In [129]:
!date

2025年  6月 19日 木曜日 13:06:20 JST
